# Import packages
Use kernel "ABM_env" -- see README.

In [ ]:
from __future__ import division
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import math
import time
from tqdm import tqdm
import cProfile
import pickle
from scipy.stats import gaussian_kde
from scipy import interpolate
import pandas as pd
from math import sqrt
import scipy.stats as stats
from matplotlib.pyplot import imshow
import matplotlib.colors as mcolors
from multiprocessing import Pool
import multiprocessing as mp
import time
import math
import matplotlib.ticker as ticker

from ABM import SEIR_multiple_pops
from amcmc import ammcmc
import os
from CalibrationMethod1_2D_methods import *

# Load training data

In [ ]:
data_file = open('./Data/Training Data/Two-parameter case/Calibration method 1/new_I_data_Two-Pop-Disc.pickle', "rb")
data = pickle.load(data_file)
data_file.close()

#Change data type to save memory:
print('Size before changing data type: ', data.nbytes)

if np.max(data) < 256:
    data = data.astype('uint8')
elif np.max(data) < 65536:
    data = data.astype('uint16')
    
print('Size after changing data type: ', data.nbytes)

#Load in mobility and jumping probability values:
parameter_matrix = pd.read_csv('./Data/Training Data/Two-parameter case/Calibration method 1/variable_parameter_values_Two-Pop-Discrete-Sample.csv', index_col=0).to_numpy()
mobilities = parameter_matrix[:,0]
jumping_probs = parameter_matrix[:,1]
random_seeds = parameter_matrix[:,2]

#Reshape to sort by mobility and jumping prob:
#based on assumption that data is given in shape (num_of_sims, num_of_time_steps, num_of_subpops)
#and that within the list of simulations, the mobility is the "outer" index (changes more slowly)
#and the jumping probability is the "inner" index (changes more quickly)

unique_mobilities = np.unique(mobilities) 
unique_jumping_probs = np.unique(jumping_probs)

num_of_mobilities = unique_mobilities.shape[0]
num_of_jumping_probs = unique_jumping_probs.shape[0]

num_of_random_seeds_per_param_set = round(data.shape[0]/(num_of_mobilities*num_of_jumping_probs)) #assuming equal number of random seeds run for each param set
num_of_time_steps = data.shape[1]
num_of_subpops = 2 #assumes 2 sub-populations

data_sorted = np.zeros((num_of_mobilities, num_of_jumping_probs, num_of_random_seeds_per_param_set, num_of_time_steps, num_of_subpops))

data_raveled = np.ravel(data, order = 'C')
data_sorted = np.reshape(data_raveled, data_sorted.shape)

#Load in constant parameter values, use to create time vector:
const_parameter_matrix = pd.read_csv('./Data/Training Data/Two-parameter case/Calibration method 1/constant_parameters_values_Two-Pop-Discrete-Sample.csv', index_col=0)
T = const_parameter_matrix['Total time (T)'][0]
del_t = const_parameter_matrix['Time step (del_t)'][0]
time_vec = np.linspace(0,T,int(T/del_t)+1)


In [ ]:
# Sum training dataset along time segments

num_of_intervals = 5 #number of time intervals to split up for likelihood calculation

intervals = np.arange(0,num_of_time_steps,int(num_of_time_steps/num_of_intervals))
intervals[-1] = intervals[-1]+num_of_time_steps%num_of_intervals
new_I_per_interval = np.zeros((num_of_intervals, num_of_mobilities, num_of_jumping_probs, num_of_random_seeds_per_param_set, num_of_subpops))

for i in range(num_of_intervals):
    data_ = data_sorted[:,:,:,intervals[i]:intervals[i+1],:]
    new_I_per_interval[i,:,:,:,:] = np.sum(data_, axis = 3)

# Check training data visually:

In [ ]:
# Check data visually:
mob_ind = 8
jump_ind = 7
rand_ind = 0
subpop_ind = 0


#--------- show new infections per time step raw data and stair plot of summed data -----------

fig, ax = plt.subplots(constrained_layout=True, dpi = 300, figsize = (4,3))
ax.plot(time_vec,data_sorted[mob_ind, jump_ind, rand_ind, :, subpop_ind], zorder = 1, label = 'New infections per time step')
ax.set_ylim(0)
ax.set_ylabel('New infections')

ax2 = ax.twinx()
ax2.stairs(new_I_per_interval[:, mob_ind, jump_ind, rand_ind, subpop_ind], np.array(intervals)/10, fill = True, color = 'grey', alpha = 0.5, zorder = -1, label = 'Total new infections per interval')
ax2.set_ylabel('Total new infections per interval')
fig.legend(bbox_to_anchor=(0.87, 0))
ax.set_xlabel('Time (days)')
plt.show()


In [ ]:
time_segment = 0
jump_ind = 7

subpop_ind = 0

plt.figure(dpi = 200)
plt.hist(new_I_per_interval[time_segment, 0, jump_ind, :, subpop_ind], alpha = 0.75, label = 'mobility = 0.0005')
plt.hist(new_I_per_interval[time_segment, 8, jump_ind, :, subpop_ind], alpha = 0.75, label = 'mobility = 0.0015')
plt.hist(new_I_per_interval[time_segment, 16, jump_ind, :, subpop_ind], alpha = 0.75, label = 'mobility = 0.0025')

plt.title('SUBPOP1: Total number of new infections during time interval '+ str(time_segment))
plt.xlabel('Total number of new infections during time interval '+ str(time_segment))
plt.legend()
plt.ylabel('Frequency')

subpop_ind = 1

plt.figure(dpi = 200)
plt.hist(new_I_per_interval[time_segment, 0, jump_ind, :, subpop_ind], alpha = 0.75, label = 'mobility = 0.0005')
plt.hist(new_I_per_interval[time_segment, 8, jump_ind, :, subpop_ind], alpha = 0.75, label = 'mobility = 0.0015')
plt.hist(new_I_per_interval[time_segment, 16, jump_ind, :, subpop_ind], alpha = 0.75, label = 'mobility = 0.0025')

plt.title('SUBPOP2: Total number of new infections during time interval '+ str(time_segment))
plt.xlabel('Total number of new infections during time interval '+ str(time_segment))
plt.legend()
plt.ylabel('Frequency')


# Generate expanded kernel density estimate PDFs for each time segment and mobility value

In [ ]:
#-------------------------PARAMETER DEFINITIONS----------------------

shape_param = 0.4
type_ = "linear"

#--------------------------NON-EXAPNDED KDE & MEANS, VARIANCES, STDEVS------------------------------

#generate kernel density estimate PDFs for each time segment and mobility value
means = np.zeros((num_of_intervals, num_of_mobilities, num_of_jumping_probs, num_of_subpops))
variances = np.zeros((num_of_intervals, num_of_mobilities, num_of_jumping_probs, num_of_subpops))
stdevs = np.zeros((num_of_intervals, num_of_mobilities, num_of_jumping_probs, num_of_subpops))
stdevs_approx = np.zeros((num_of_intervals, num_of_mobilities, num_of_jumping_probs, num_of_subpops)) #approximate stdev for 0-stdev pdfs based on replacement approximation function

KDE_fns = np.zeros((num_of_intervals, num_of_mobilities, num_of_jumping_probs, num_of_subpops))
KDE_fns = KDE_fns.astype('object')

for interval in tqdm(range(num_of_intervals)):
    for mob_ind in range(num_of_mobilities):
        for jump_ind in range(num_of_jumping_probs):
            for subpop_ind in range(num_of_subpops):
                means[interval, mob_ind, jump_ind, subpop_ind] = np.mean(new_I_per_interval[interval, mob_ind, jump_ind, :, subpop_ind])
                variances[interval, mob_ind, jump_ind, subpop_ind] = np.var(new_I_per_interval[interval, mob_ind, jump_ind, :, subpop_ind])
                stdevs[interval, mob_ind, jump_ind, subpop_ind] = np.std(new_I_per_interval[interval, mob_ind, jump_ind, :, subpop_ind])
                stdevs_approx[interval, mob_ind, jump_ind, subpop_ind] = np.std(new_I_per_interval[interval, mob_ind, jump_ind, :, subpop_ind])
                try: 
                    kde = gaussian_kde(new_I_per_interval[interval, mob_ind, jump_ind, :, subpop_ind]) #generate KDE -- will fail if all values are 0's
                    KDE_fns[interval, mob_ind, jump_ind, subpop_ind] = kde.pdf
                except: 
                    mean = means[interval, mob_ind, jump_ind, subpop_ind]
                    if mean > 0:
                        print(mean)
                    KDE_fns[interval, mob_ind, jump_ind, subpop_ind], stdev_of_approx_fn = return_zero_stdev_pdf(mean, shape_param, type_)  
                    stdevs_approx[interval, mob_ind, jump_ind, subpop_ind] = stdev_of_approx_fn


# Check KDEs
Reproduces Fig. 6.

In [ ]:
#-----------------CHOOSE WHICH TIME SEGMENT AND JUMPING INDEX TO PLOT---------------
time_segment = 0
jump_ind = 7
mob_ind = 8
grid_num = 1000

#-------------------------PLOTTING----------------------
subpop_ind = 0
plt.figure(dpi = 300, figsize = (4,3))
plt.plot(np.linspace(0,100,grid_num), KDE_fns[time_segment,0,jump_ind, subpop_ind](np.linspace(0,100,grid_num)), alpha = 0.75, label = 'Mobility = 0.005')
plt.plot(np.linspace(0,100,grid_num), KDE_fns[time_segment,8,jump_ind, subpop_ind](np.linspace(0,100,grid_num)), alpha = 0.75, label = 'Mobility = 0.015')
plt.plot(np.linspace(0,100,grid_num), KDE_fns[time_segment,16,jump_ind, subpop_ind](np.linspace(0,100,grid_num)), alpha = 0.75, label = 'Mobility = 0.025')
plt.title('Sub-population 1')
plt.xlabel('Total number of new infections')
plt.legend()
plt.ylabel('Probability density')

subpop_ind = 1
plt.figure(dpi = 300, figsize = (4,3))
plt.plot(np.linspace(0,100,grid_num), KDE_fns[time_segment,0,jump_ind, subpop_ind](np.linspace(0,100,grid_num)), alpha = 0.75, label = 'Mobility = 0.005')
plt.plot(np.linspace(0,100,grid_num), KDE_fns[time_segment,8,jump_ind, subpop_ind](np.linspace(0,100,grid_num)), alpha = 0.75, label = 'Mobility = 0.015')
plt.plot(np.linspace(0,100,grid_num), KDE_fns[time_segment,16,jump_ind, subpop_ind](np.linspace(0,100,grid_num)), alpha = 0.75, label = 'Mobility = 0.025')
plt.title('Sub-population 2')
plt.xlabel('Total number of new infections')
plt.legend()
plt.ylabel('Probability density')


In [ ]:
def f_pdf(sample_point, new_I_per_interval_ref):
    """
    function that takes a sample_point [sample_mobility, sample_jump_prob] and returns posterior log-likelihood given data in new_I_per_interval_ref
    
    posterior likelihood is calculated by first calculating interpolated PDFs at the sample_point parameter values.
    then, the log-likelihood values are simply sampled directly from these PDFs based on the given data.
    """

    sample_mobility = sample_point[0]
    sample_jump_prob = sample_point[1]
    
    if sample_mobility<=0.025 and sample_mobility>=0.005: #don't like to have this hard-coded here either but also interpolation doesn't work otherwise, and computation time is wasted because prior will be 0 anyway
        if sample_jump_prob<=0.1 and sample_jump_prob>=0:
            KDE = interp_KDE_2d(sample_jump_prob, sample_mobility, means, variances, stdevs, stdevs_approx, KDE_fns,num_of_intervals, num_of_subpops)
            log_probs = np.zeros_like(KDE)
            
            for interval in range(num_of_intervals):
                for subpop_ind in range(num_of_subpops):
                    log_probs[interval, subpop_ind] = np.log(KDE[interval, subpop_ind](new_I_per_interval_ref[interval, subpop_ind]))
                
            if np.sum(log_probs) == -np.inf:
                return [-1e100,0]
            else:
                return [np.sum(log_probs),0]
    else:
        return [-1e100,0]
    

### Import test/sample data:

In [ ]:
data_file = open('./Data/Test Data/Two-parameter case/new_I_data_Two-Pop-NEW-COMBINED-TEST.pickle', "rb")
data = pickle.load(data_file)
data_file.close()

#Load in mobility and jumping probability values:
parameter_matrix = pd.read_csv('./Data/Test Data/Two-parameter case/variable_parameter_values_Two-Pop-NEW-COMBINED-TEST.csv', index_col=0).to_numpy()
mobilities = parameter_matrix[:,0]
jumping_probs = parameter_matrix[:,1]
random_seeds = parameter_matrix[:,2]
num_of_samples = mobilities.shape[0]

intervals = np.arange(0,num_of_time_steps,int(num_of_time_steps/num_of_intervals))
intervals[-1] = intervals[-1]+num_of_time_steps%num_of_intervals
new_I_per_interval = np.zeros((num_of_samples, num_of_intervals, num_of_subpops))

for i in range(num_of_intervals):
    data_ = data[:,intervals[i]:intervals[i+1],:]
    print(data_.shape)
    new_I_per_interval[:,i,:] = np.sum(data_, axis = 1)

for i in range(num_of_samples):
    if np.max(new_I_per_interval[i,:,:]) == 0:
        print('Data all zeroes')
        print('Sample ID: ', i)
    


In [ ]:
# Change nsamples, nburn, n_j, and n_m to get longer MCMC run and more detailed brute force posterior estimation

def posterior_estimation(sample_data_set_ind):
    
    new_I_per_interval_ref = new_I_per_interval[sample_data_set_ind]
    RUN_NAME = 'Two-Pop'
    
    #-------- MCMC ----------
    #test params:
    nsamples = 2000
    nburn = 2000
    seed=100

    ##Full analysis params:
    # nsamples = 75000
    # seed=100
    # nburn = 5000

#     nskip = 10000
    nskip = 0
    nthin = 1
    tmpchn_dir = './MCMC_example_results/'+RUN_NAME
    logfile_dir = "./MCMC_example_results/"+RUN_NAME
    
    os.makedirs(tmpchn_dir, exist_ok = True)
    os.makedirs(logfile_dir, exist_ok = True)

    tmpchn = "./MCMC_example_results/"+RUN_NAME+"/amcmc_TMP_ABM_sample_ind_"+str(sample_data_set_ind)+".dat"
    logfile = "./MCMC_example_results/"+RUN_NAME+"/amcmc_LOG_ABM_sample_ind_"+str(sample_data_set_ind)+".dat"
    if os.path.isfile(tmpchn): #remove previous run if it exists
        os.remove(tmpchn)
    if os.path.isfile(logfile): #remove previous run if it exists
        os.remove(logfile)
    opts = {"nsteps": nsamples, "nfinal": 10000000,"gamma": 1,
            "inicov": np.array([[1E-3,0],[0,1E-3]]),"inistate": np.array([0.015, 0.0005]),
            "spllo": np.array([0.005,0]),"splhi": np.array([[0.025,0.001]]),
            "logfile": logfile,"burnsc":5,
            "nburn":nburn,"nadapt":100,"coveps":1.e-10,"ofreq":50,"tmpchn":tmpchn,'rnseed':int(sample_data_set_ind)
            }

    ndim = 2
    np.random.seed(seed)
    theta0 = np.array([0.0151, 0.00051])
    print(f'Start point:{theta0}')
    opts["inistate"] = theta0

    print('Sampling likelihood function with AMCMC ...')
    start_time = time.time()
    sol=ammcmc(opts,f_pdf,new_I_per_interval_ref)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed AMCMC Run Time: {elapsed_time} seconds")

    samples = sol['chain']
    logprob = sol['minfo'][:,1]

    import matplotlib.pyplot as plt
    samples = samples[nskip::nthin]
    logprob = logprob[nskip::nthin]

    print('Acceptance rate',sol['accr'])
    print('Mean:',np.mean(samples, axis=0))
    print('Var:',np.var(samples, axis=0))
    print('Cov:',np.cov(samples.T)) 

    #save data:
    with open('./MCMC_example_results/'+RUN_NAME+'/AMCMC_sample_ind_'+str(int(sample_data_set_ind))+'.pickle', 'wb') as handle:
        pickle.dump(sol, handle, protocol=pickle.HIGHEST_PROTOCOL)

    #----------Brute force sampling----------
    save_dir = './Brute_force_posterior_estimation_example/'+RUN_NAME
    os.makedirs(save_dir, exist_ok = True)
    save_file = save_dir+'/new_I_data_'+RUN_NAME+'-IND'+str(int(sample_data_set_ind))+'LOG_PROBS.txt'
    
    if os.path.isfile(save_file): #remove previous run if it exists
        os.remove(save_file)

    # n_j = 75
    # n_m = 76
    n_j = 10
    n_m = 10
    test_jump_probs = np.linspace(unique_jumping_probs[0]+1E-10, unique_jumping_probs[-1]-1E-10, n_j)
    test_mobilities = np.linspace(unique_mobilities[0]+1E-10, unique_mobilities[-1]-1E-10, n_m)

    log_probs = np.zeros((test_jump_probs.shape[0], test_mobilities.shape[0]))
    tmparg = 0

    start_time = time.time()
    for i in tqdm(range(test_jump_probs.shape[0])):
        for j in range(test_mobilities.shape[0]):
            log_probs[i,j] = f_pdf([test_mobilities[j], test_jump_probs[i]], new_I_per_interval_ref)[0]
        
            fout = open(save_file, 'ab')
            dataout = np.array([[test_mobilities[j],test_jump_probs[i],log_probs[i,j]]])
            np.savetxt(fout, dataout, fmt='%.8e',delimiter=' ', newline='\n')
            fout.close()

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed brute force posterior estimation run time: {elapsed_time} seconds")

    with open(save_dir+'/new_I_data_'+RUN_NAME+'-IND'+str(int(sample_data_set_ind))+'LOG_PROBS.pickle', 'wb') as handle:
        pickle.dump(log_probs, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

In [ ]:
sample_data_set_ind = 1500
posterior_estimation(sample_data_set_ind)

## Plotting

## MCMC figure generation

In [ ]:
#--------------Specify batch posterior estimation folder, nburn, sample_id-----------------
MCMC_res_folder = './MCMC_example_results/Two-Pop/'
nburn = 0
sample_id = 1500

#------------- Load in mobility and jumping probability values ------------:
parameter_matrix = pd.read_csv('./Data/Test Data/Two-parameter case/variable_parameter_values_Two-Pop-NEW-COMBINED-TEST.csv', index_col=0).to_numpy() 
mobilities = parameter_matrix[:,0]
jumping_probs = parameter_matrix[:,1]
random_seeds = parameter_matrix[:,2]
num_of_samples = mobilities.shape[0]

completed_samples = []
incomplete_samples = []
within_95_conf_int_samples = np.zeros(1666)
within_50_conf_int_samples = np.zeros(1666)

    
print('-------------------------')
print('-------Sample '+str(sample_id)+'-------')
print('-------------------------')

actual_mob = mobilities[sample_id]
actual_jp = jumping_probs[sample_id]


# -------- set up colormaps -----------
color_1 = (237/255,248/255,251/255)
color_2 = (35/255,139/255,69/255)
colors = [color_1, color_2]
my_colormap = mcolors.LinearSegmentedColormap.from_list("CustomColormap", colors)

#-------------------------------------------------
#-----------Load and plot MCMC results------------
#-------------------------------------------------
# try:
#Load MCMC results:
MCMC_file = MCMC_res_folder+'AMCMC_sample_ind_'+str(sample_id)+'.pickle'
data_file = open(MCMC_file, "rb")
sol = pickle.load(data_file)
data_file.close()

# -------- calculate marginal CI boundaries -----------

#Calculate marginal 95% CI for mobility
upper_95_CI_mob = np.percentile(sol['chain'][nburn:,0], 97.5)
lower_95_CI_mob = np.percentile(sol['chain'][nburn:,0], 2.5)

#Calculate marginal 95% CI for jumping prob
upper_95_CI_jp = np.percentile(sol['chain'][nburn:,1], 97.5)
lower_95_CI_jp = np.percentile(sol['chain'][nburn:,1], 2.5)

#Calculate marginal 95% CI for mobility
upper_50_CI_mob = np.percentile(sol['chain'][nburn:,0], 75)
lower_50_CI_mob = np.percentile(sol['chain'][nburn:,0], 25)

#Calculate marginal 95% CI for jumping prob
upper_50_CI_jp = np.percentile(sol['chain'][nburn:,1], 75)
lower_50_CI_jp = np.percentile(sol['chain'][nburn:,1], 25)

#Calculate 50% and 95% confidence interval bounds:
Z, inside_95, inside_hollow_95, Z_renormalized_95, Z_marked_95, x_flat, y_flat = MCMC_KDE(sol, nburn = nburn, target = 0.95)
#     within_confidence_int = check_if_within_confidence_int([actual_mob, actual_jp], x_flat, y_flat, inside)
Z, inside_50, inside_hollow_50, Z_renormalized_50, Z_marked_50, x_flat, y_flat = MCMC_KDE(sol, nburn = nburn, target = 0.5)

#----------------Plot scatter plot of chain------------------
plt.figure(dpi = 500, figsize = (3.5,3.5))
plt.scatter(sol['chain'][nburn:,1], sol['chain'][nburn:,0], s=1, color = (102/255,194/255,164/255), label = 'MCMC chain')
plt.ylim(0.005,0.025)
plt.xlim(0,0.001)
plt.scatter(actual_jp, actual_mob, color = 'blue', marker = '*', label = 'True parameter')
plt.ylabel('Mobility')
plt.xlabel('Jumping probability')
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
#     plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.legend()
#     plt.savefig('figs_final/2d_MCMC_results/chain-sample'+str(sample_id)+'.png',bbox_inches='tight')
plt.show()

#----------------Posterior plot with 95 and 50% conf intervals (WITH colorbar) ------------------
colors = [(0.9,0,0,c) for c in np.linspace(0,1,100)]
cmapred = mcolors.LinearSegmentedColormap.from_list('mycmap', colors, N=2)
colors = [(0.9,0.6,0.6,c) for c in np.linspace(0,1,100)]
cmapblue = mcolors.LinearSegmentedColormap.from_list('mycmap', colors, N=2)

fig, ax = plt.subplots(dpi = 300)
ax_main = plt.subplot()
pcm = ax_main.pcolormesh(y_flat, x_flat, Z.T, cmap=my_colormap)
ax_main.pcolormesh(y_flat, x_flat, inside_hollow_95.T,shading='auto',cmap=cmapblue,label='95% CI')
ax_main.pcolormesh(y_flat, x_flat, inside_hollow_50.T,shading='auto',cmap=cmapred,label='50% CI')
plt.xlabel('Jumping probability')
plt.ylabel('Mobility')
plt.scatter(actual_jp, actual_mob, c='blue', label = 'True parameter', marker ='*')
d = []
plt.plot(d,d,linewidth=2,color = (0.9,0,0), label='95% CI')
plt.plot(d,d,linewidth=2,color = (0.9,0.6,0.6), label='50% CI')
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
#     plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.legend()

ax_cbar = plt.subplot()
cax = make_square_axes_with_colorbar(ax=ax_cbar, size=0.15, pad=0.1)
cbar = fig.colorbar(pcm, cax=cax)
cbar.set_label('Probability density')

#     cbar.set_ticks([])  # Remove tick labels from the colorbar

# plt.savefig('figs_final/2d_MCMC_results/posterior-with-colorbar-sample'+str(sample_id)+'.png',bbox_inches='tight')

plt.show()

#----------------Posterior plot with no confidence intervals (WITH colorbar) ------------------
colors = [(0.9,0,0,c) for c in np.linspace(0,1,100)]
cmapred = mcolors.LinearSegmentedColormap.from_list('mycmap', colors, N=2)
colors = [(0.9,0.6,0.6,c) for c in np.linspace(0,1,100)]
cmapblue = mcolors.LinearSegmentedColormap.from_list('mycmap', colors, N=2)

fig, ax = plt.subplots(dpi = 500, figsize = (3.5,3.5))
ax_main = plt.subplot()
pcm = ax_main.pcolormesh(y_flat, x_flat, Z.T, cmap=my_colormap)
plt.xlabel('Jumping probability')
plt.ylabel('Mobility')
plt.ylim(0.005,0.025)
plt.xlim(0,0.001)
plt.scatter(actual_jp, actual_mob, c='blue', label = 'True parameter', marker ='*')
d = []
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
#     plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

#     plt.legend()

exp_value = 5 #MANUALLY SET THE EXP VALUE FOR SCI NOTATION ON COLORBAR

def fmt(x, pos):
    return str(x/(10**exp_value))

ax_cbar = plt.subplot()
# cax = make_square_axes_with_colorbar(ax=ax_cbar, size=0.15, pad=0.1)
cbar = fig.colorbar(pcm, cax=cax, format=ticker.FuncFormatter(fmt))
cbar.set_label('Probability density')

# # Adding exponent value at the top
# exp_value = int(np.log10(ticks[-1]))
cbar.ax.text(1.05, 1, f"1e{exp_value}", transform=cbar.ax.transAxes, va='bottom', ha='left')
#     cbar.set_ticks([])  # Remove tick labels from the colorbar

#     plt.savefig('figs_final/2d_MCMC_results/posterior-no-CI_sample'+str(sample_id)+'.png',bbox_inches='tight')

plt.show()

# ----------------- Marginal plots --------------------
mob_marginal = np.trapz(Z, y_flat, axis = 0)
jp_marginal = np.trapz(Z, x_flat, axis = 1)

print('mob marginal integral', np.trapz(mob_marginal, x_flat))
print('jp marginal integral', np.trapz(jp_marginal, y_flat))

#jumping probability marginal:
plt.figure(figsize = (3.5, 1), dpi = 500)
plt.plot(y_flat, jp_marginal, linewidth=2, color = color_2)
plt.xlim(0,0.001)
plt.axvline(actual_jp, linewidth=2, color = "blue", linestyle = "--") #true param
plt.axvline(upper_95_CI_jp, linewidth=2, color = (0.9,0,0))
plt.axvline(lower_95_CI_jp, linewidth=2, color = (0.9,0,0))
plt.axvline(upper_50_CI_jp, linewidth=2, color = (0.9,0.6,0.6))
plt.axvline(lower_50_CI_jp, linewidth=2, color = (0.9,0.6,0.6))
#     plt.savefig('figs_final/2d_MCMC_results/JP_marginal_posterior-sample'+str(sample_id)+'.png',bbox_inches='tight')
plt.show()

#mobility marginal:
plt.figure(figsize = (3.5, 1), dpi = 500)
plt.scatter([], [], c='blue', marker="*", label='True parameter')
plt.axvline(actual_mob, linewidth=2, color = "blue", label='True parameter', linestyle = "--") #true param
plt.plot(x_flat, mob_marginal, linewidth=2, color = color_2, label = 'Marginal posterior')
plt.xlim(0.025,0.005)
plt.axvline(upper_95_CI_mob, linewidth=2, color = (0.9,0,0), label='Marginal 95% CI')
plt.axvline(lower_95_CI_mob, linewidth=2, color = (0.9,0,0))
plt.axvline(upper_50_CI_mob, linewidth=2, color = (0.9,0.6,0.6), label='Marginal 50% CI')
plt.axvline(lower_50_CI_mob, linewidth=2, color = (0.9,0.6,0.6))
plt.xticks(np.linspace(0.0050,0.025,9))
plt.legend(bbox_to_anchor=(1.3, 0.95))
#     plt.savefig('figs_final/2d_MCMC_results/MOB_marginal_posterior-sample'+str(sample_id)+'.png',bbox_inches='tight')

plt.show()

# ----------------- Range checking and print outs ------------------

interp_95=scipy.interpolate.RegularGridInterpolator((y_flat,x_flat), inside_95, method='nearest')
interp_50=scipy.interpolate.RegularGridInterpolator((y_flat,x_flat), inside_50, method='nearest')

within_95_conf_int_samples[sample_id] = interp_95((actual_jp, actual_mob))
within_50_conf_int_samples[sample_id] = interp_50((actual_jp, actual_mob))

print('95% confidence interval:', within_95_conf_int_samples[sample_id])
print('50% confidence interval:', within_50_conf_int_samples[sample_id])

# except:
#     print('Issue with MCMC')



In [ ]:

#------------------------------------------------------------------
#-----------Load and plot brute force posterior results------------
#------------------------------------------------------------------
file_brute = './Brute_force_posterior_estimation_example/Two-Pop/new_I_data_Two-Pop-IND'+str(sample_id)+'LOG_PROBS.txt'  #Combined first and second batch of MCMC

brute_posterior = pd.read_csv(file_brute, header=None, sep=" ").to_numpy()
x_flat = np.unique(brute_posterior[:,0])
y_flat = np.unique(brute_posterior[:,1])[:-1]
brute_posterior_truncate = brute_posterior[:y_flat.shape[0]*x_flat.shape[0]]
Z = np.exp((brute_posterior_truncate[:,2]).reshape((y_flat.shape[0], x_flat.shape[0])))

#----------------Plot with pcolormesh: likelihood (WITH colorbar)------------------
fig, ax = plt.subplots(dpi = 500,  figsize = (4.25,4.25))

ax_main = plt.subplot()
pcm = ax_main.pcolormesh(y_flat, x_flat, Z.T, cmap=my_colormap)
ax_main.set_xlabel('Jumping probability')
ax_main.set_ylabel('Mobility')
plt.ylim(0.005,0.025)
plt.xlim(0,0.001)
ax_main.scatter(jumping_probs[sample_id], mobilities[sample_id], c='blue', marker="*", label='True parameter')
ax_main.legend()
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
# plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

#------ new colorbar code (scientific notation on tick marks) =========
import matplotlib.ticker as ticker

exp_value = -4 #MANUALLY SET THE EXP VALUE FOR SCI NOTATION ON COLORBAR

def fmt(x, pos):
    return str(round(x/(10**exp_value), 2))

ax_cbar = plt.subplot()
cax = make_square_axes_with_colorbar(ax=ax_cbar, size=0.15, pad=0.1)
cbar = fig.colorbar(pcm, cax=cax, format=ticker.FuncFormatter(fmt))
cbar.set_label('Likelihood')

# # Adding exponent value at the top
# exp_value = int(np.log10(ticks[-1]))
cbar.ax.text(1.05, 1, f"1e{exp_value}", transform=cbar.ax.transAxes, va='bottom', ha='left')

# Adjust layout to prevent overlap of labels and colorbar
plt.tight_layout()
# plt.savefig('figs_final/2d_MCMC_results/likelihood-with-colorbar-sample'+str(sample_id)+'.png',bbox_inches='tight')

plt.show()

#----------------Plot with pcolormesh: posterior (WITH colorbar)------------------

#normalize to get posterior from likelihood:
integral = np.trapz(Z, y_flat, axis = 0)
integral = np.trapz(integral, x_flat, axis = 0)
print(integral)
Z = Z/integral
# Z[10,:] = 100000 #testing visualization direction

fig, ax = plt.subplots(dpi = 500,  figsize = (4.25,4.25))

ax_main = plt.subplot()
pcm = ax_main.pcolormesh(y_flat, x_flat, Z.T, cmap=my_colormap)
ax_main.set_xlabel('Jumping probability')
ax_main.set_ylabel('Mobility')
plt.ylim(0.005,0.025)
plt.xlim(0,0.001)
ax_main.scatter(jumping_probs[sample_id], mobilities[sample_id], c='blue', marker="*", label='True parameter')
ax_main.legend()
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
# plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

#------ new colorbar code (scientific notation on tick marks) =========
import matplotlib.ticker as ticker

exp_value = 5 #MANUALLY SET THE EXP VALUE FOR SCI NOTATION ON COLORBAR

def fmt(x, pos):
    return str(round(x/(10**exp_value), 2))

ax_cbar = plt.subplot()
# cax = make_square_axes_with_colorbar(ax=ax_cbar, size=0.15, pad=0.1)
cbar = fig.colorbar(pcm, cax=cax, format=ticker.FuncFormatter(fmt))
cbar.set_label('Probability density')

# # Adding exponent value at the top
# exp_value = int(np.log10(ticks[-1]))
cbar.ax.text(1.05, 1, f"1e{exp_value}", transform=cbar.ax.transAxes, va='bottom', ha='left')

# Adjust layout to prevent overlap of labels and colorbar
plt.tight_layout()
# plt.savefig('figs_final/2d_MCMC_results/grid-sampled-posterior-with-colorbar-sample'+str(sample_id)+'.png',bbox_inches='tight')

plt.show()

